In [22]:
import numpy as np
from matplotlib import pyplot as plt
import DSPPythonWrapper as wrp


In [23]:
mesh_width = 45.00  # mm
mesh_height = 30.00  # mm
spatial_resolution = 5  # mm

mesh = wrp.Triangular2DMesh(mesh_width, mesh_height,
    spatial_resolution)

mesh_coords = mesh.GetMeshCoordinates()
# How large should this vector be?
lenk_internal = np.ceil(mesh_width * 2 / spatial_resolution)
lenk_internal += 1 - np.mod(lenk_internal, 2)
expected_c = np.ceil(mesh_height / (spatial_resolution
    * np.sqrt(3) / 2))
expected_c += np.mod(expected_c, 2)
expected_k = lenk_internal // 2 + 1

assert(expected_c == mesh_coords.shape[0])
assert(expected_k == mesh_coords.shape[1])
assert(2 == mesh_coords.shape[2])
assert(mesh_coords.dtype == np.float32)
print("All assertions passed")

All assertions passed
